In [1]:
import os

In [2]:
%pwd

'c:\\code\\ML\\time_series_forecast\\time-series-forecast\\research'

In [3]:
os.chdir('../')

In [14]:
%pwd

'c:\\code\\ML\\time_series_forecast\\time-series-forecast'

In [8]:
pip install pandas


  Using cached pandas-2.3.0-cp310-cp310-win_amd64.whl (11.1 MB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
#entity 
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataIngestionconfig:
  root_dir : Path
  source_url : str
  local_data_file : Path

In [18]:
# configuration
from src.time_series.constants import *
from src.time_series.utils.common import create_directories , read_yaml

In [19]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionconfig:
        config = self.config.data_ingestion
        data_ingestion_config = DataIngestionconfig(root_dir= config.root_dir,source_url=config.source_url,local_data_file=config.local_data_file)
        return data_ingestion_config

In [20]:
#components

In [22]:
import os
import urllib.request 
import zipfile
from src.time_series.logger import logger
import requests
class DataIngestionComponent:
    def __init__(self,config:DataIngestionconfig):
        self.config = config

    def download_file(self):
        url=self.config.source_url
        output_path=self.config.local_data_file
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        logger.info(f"Starting download from {self.config.source_url}...")
        try:
            with requests.get(url, stream=True) as r:
                r.raise_for_status()
                with open(output_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
            print(f"File successfully downloaded to: {output_path}")
        except Exception as e:
            print(f"Download failed: {e}")
        return self.config.local_data_file

In [23]:
#pipeline

In [25]:
try :
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion_comp = DataIngestionComponent(config=data_ingestion_config)
    data_ingestion_comp.download_file()
except Exception as e:
    raise e

[2025-07-02 21:08:37,644: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-02 21:08:38,028: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-02 21:08:38,045: INFO: common: created directory at: artifacts]
[2025-07-02 21:08:38,053: INFO: 3277893088: Starting download from https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv...]
File successfully downloaded to: artifacts/data_ingestion/airline.csv
